<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/238_PredRevenue_Gap_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scoring Utilities for Revenue Gap Orchestrator


# 🧠 **What the Scoring Layer Actually Is**

This file teaches one of the most valuable concepts in orchestrator design:

> **Raw gaps ≠ intelligence.
> Scoring *is* the intelligence.**

Gap detection tells you *what’s wrong.*
Scoring tells you *what matters.*

Think of it like triage in an ER:

* Doctors detect injuries (gap detection)
* Nurses rank patients in order of urgency (scoring)

This is why this file is so powerful — it turns raw detections into **prioritized decisions**.

---

# 🧱 **Overview: The Four Dimensions of Scoring**

Every gap gets evaluated across **four lenses**:

1. **Revenue Impact**
   → How much money are we losing?

2. **Churn Risk**
   → How likely is this customer to leave completely?

3. **Customer Value**
   → If we save them, how important are they?

4. **Recovery Probability**
   → If we act, do we have a reasonable chance of fixing it?

Then the orchestration pipeline uses weighted aggregation to produce a final score.

This transforms chaotic data → strategic action.

---

# 🔍 **Let’s break down each scoring dimension — with the lessons behind them.**

---

# 1️⃣ Revenue Impact Score

**Function:** `calculate_revenue_impact_score`

**Goal:** Measure *how big* the revenue gap is, on a clean 0–10 scale.

### 🔑 Key Concepts to Learn

* **Normalization**
  Gap amounts vary wildly (a \$200 drop vs. a \$2,000 drop).
  So the system normalizes every gap against the **largest gap in the dataset** to keep scores proportional.

* **Comparability**
  A revenue gap score of 9.5 *always* means “big problem” regardless of the company's size.

### Why it matters

You can't prioritize effectively unless every gap is on the same scale.

---

# 2️⃣ Churn Risk Score

**Function:** `calculate_churn_risk_score`

This is one of the smartest parts of the entire architecture.

### 🔑 What you're learning here

There are **two sources of churn risk**:

1. **Gap-related churn risk**
   e.g., declining revenue, zero spend

2. **Standalone churn risk signal**
   e.g., customer has 3+ zero-spend weeks
   (computed from the churn detection utilities)

### Logic overview:

* Zero spend → **very high churn risk**
* Declining revenue → **moderate churn risk**
* Below baseline → **slight churn risk**
* Independent churn detection → overrides with a clearer score

### Why this is brilliant

It blends:

* *Symptomatic churn risk* (detected through gaps)
* *Physiological churn risk* (detected from pure behavior signals)

This is exactly how CRM and retention systems work in large companies.

---

# 3️⃣ Customer Value Score

**Function:** `calculate_customer_value_score`

This answers the question:

> **"If we save this customer, how big is the long-term payoff?"**

### 🔑 What you're learning

* **Percentile ranking**
  Customers are sorted by lifetime/12-week revenue.
  Percentiles are translated to a 0–10 score.

* **Economic prioritization**
  Top 10% spenders = score 10
  Bottom 10% = score 0–2

This forces the orchestrator to:

* Care more about important customers
* Not waste interventions on low-value ones

This is classic **customer economics**, built into your agent!

---

# 4️⃣ Recovery Probability Score

**Function:** `calculate_recovery_probability_score`

This is the “return on effort” calculator.

### 🔑 Key ideas

* **Zero spend is hard to reverse**
  (score: 2–6 depending on zero-week duration)

* **Declining revenue can be reversible**
  (score: 5–7)

* **Below baseline gaps are the easiest to recover**
  (score: 7)

* **Long-term gaps decrease score**
  (duration penalty)

### Why this is genius

Even if a customer has a big gap,
it may not make sense to prioritize them
if the probability of fixing them is too low.

---

# 🧮 Step 5: Weighted Aggregation — the real orchestrator intelligence

**Function:** `score_gap`

This is where the magic happens.

### Weighted formula:

```
final_score =
  revenue_impact * 0.35 +
  churn_risk * 0.30 +
  customer_value * 0.20 +
  recovery_probability * 0.15
```

You should study and understand:

### ✔ The weights reflect business strategy

* Revenue impact is most important
* Churn risk almost as important
* Customer value moderately important
* Recovery probability adds realism

### ✔ Changing the weights changes the personality of the agent

* Increase churn risk weight → more focus on retention
* Increase revenue impact weight → more focus on large high-value accounts
* Increase recovery probability → more conservative prioritization

This is how you customize the intelligence of an orchestrator.

---

# 🏆 Step 6: Ranking & Summaries

**Function:** `rank_gaps`

This produces the *actual business output*:

* Ranked list of all gaps
* Top N actionable items
* A summary with:

  * Total revenue gap
  * High/medium/low severity counts
  * Potential revenue recovery

This turns your agent from a **tool** into a **business decision engine.**

---

# 🧠 What You Should Take Away (Most Important Lessons)

### ✔ Scoring is where detection becomes prioritization

This is the step that makes your agent *useful*, not just informative.

### ✔ Each score is independent and modular

You can:

* swap ML models
* add new dimensions
* adjust weights
* or use dynamic scoring based on context
  without rewriting the agent.

### ✔ Scoring is subjective — and intentionally configurable

Different companies → different scoring strategies.

### ✔ The scoring system reflects business strategy

You’re learning how to encode:

* retention strategies
* revenue protection policies
* customer value hierarchies
  into a single unified pipeline.

This is the core skill of building real production orchestrators.



In [ ]:
"""
Scoring Utilities for Revenue Gap Orchestrator

Score gaps by revenue impact, churn risk, customer value, and recovery probability.
"""

from typing import Dict, List, Any, Optional


def calculate_revenue_impact_score(
    gap: Dict[str, Any],
    max_gap_amount: float = 1000.0
) -> float:
    """
    Calculate revenue impact score (0-10 scale).

    Args:
        gap: Gap dictionary
        max_gap_amount: Maximum gap amount for normalization

    Returns:
        Score from 0-10
    """
    gap_amount = abs(gap.get("gap_amount", 0.0))

    # Normalize to 0-10 scale
    if max_gap_amount > 0:
        score = min(10.0, (gap_amount / max_gap_amount) * 10.0)
    else:
        score = 0.0

    return round(score, 2)


def calculate_churn_risk_score(
    gap: Dict[str, Any],
    churn_risk_data: Optional[Dict[str, Any]] = None
) -> float:
    """
    Calculate churn risk score (0-10 scale).

    Args:
        gap: Gap dictionary
        churn_risk_data: Churn risk data for the customer (if available)

    Returns:
        Score from 0-10
    """
    # Check gap type for churn indicators
    gap_type = gap.get("gap_type", "")
    weeks_at_risk = gap.get("weeks_at_risk", 0)

    # Zero spend gaps are high churn risk
    if gap_type == "zero_spend":
        # More weeks = higher risk
        score = min(10.0, 7.0 + (weeks_at_risk * 1.0))
    elif churn_risk_data:
        # Use churn risk score if available
        churn_score = churn_risk_data.get("churn_risk_score", 0.0)
        score = churn_score * 10.0
    elif gap_type in ["declining_revenue", "below_baseline"]:
        # Declining revenue indicates some churn risk
        gap_percentage = abs(gap.get("gap_percentage", 0.0))
        score = min(10.0, (gap_percentage / 50.0) * 10.0)  # 50% gap = max risk
    else:
        score = 0.0

    return round(score, 2)


def calculate_customer_value_score(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    all_customers_baselines: Dict[str, Dict[str, Any]]
) -> float:
    """
    Calculate customer value score (0-10 scale).

    Args:
        customer_id: Customer ID
        revenue_baseline: Revenue baseline for this customer
        all_customers_baselines: All customers' baselines (for comparison)

    Returns:
        Score from 0-10
    """
    if not all_customers_baselines:
        return 5.0  # Default middle score

    # Get customer's total revenue
    customer_total = revenue_baseline.get("total_revenue", 0.0)

    # Calculate percentile
    all_totals = [
        baseline.get("total_revenue", 0.0)
        for baseline in all_customers_baselines.values()
    ]

    if not all_totals:
        return 5.0

    # Find percentile
    sorted_totals = sorted(all_totals)
    customer_rank = sum(1 for total in sorted_totals if total <= customer_total)
    percentile = (customer_rank / len(sorted_totals)) * 100

    # Convert percentile to 0-10 score
    # Top 10% = 10, bottom 10% = 0
    if percentile >= 90:
        score = 10.0
    elif percentile >= 75:
        score = 8.0
    elif percentile >= 50:
        score = 6.0
    elif percentile >= 25:
        score = 4.0
    else:
        score = 2.0

    return round(score, 2)


def calculate_recovery_probability_score(
    gap: Dict[str, Any],
    revenue_baseline: Dict[str, Any]
) -> float:
    """
    Calculate recovery probability score (0-10 scale).

    Higher score = more likely to recover.

    Args:
        gap: Gap dictionary
        revenue_baseline: Revenue baseline data

    Returns:
        Score from 0-10
    """
    gap_type = gap.get("gap_type", "")
    weeks_at_risk = gap.get("weeks_at_risk", 0)
    revenue_trend = revenue_baseline.get("revenue_trend", "stable")

    # Base score on gap type and duration
    if gap_type == "zero_spend":
        # Zero spend is harder to recover from
        if weeks_at_risk >= 3:
            score = 2.0  # Very low recovery probability
        elif weeks_at_risk == 2:
            score = 4.0
        else:
            score = 6.0
    elif gap_type == "declining_revenue":
        # Declining revenue - check trend
        if revenue_trend == "declining":
            score = 5.0  # Moderate recovery probability
        else:
            score = 7.0
    elif gap_type == "below_baseline":
        # Below baseline - easier to recover
        score = 7.0
    else:
        score = 5.0  # Default

    # Adjust based on weeks at risk
    if weeks_at_risk > 4:
        score *= 0.7  # Reduce score for long-term gaps
    elif weeks_at_risk > 2:
        score *= 0.85

    return round(max(0.0, min(10.0, score)), 2)


def score_gap(
    gap: Dict[str, Any],
    churn_risk_data: Optional[Dict[str, Any]],
    revenue_baseline: Dict[str, Any],
    all_customers_baselines: Dict[str, Dict[str, Any]],
    scoring_weights: Dict[str, float],
    max_gap_amount: float = 1000.0
) -> Dict[str, Any]:
    """
    Score a gap using all dimensions.

    Args:
        gap: Gap dictionary
        churn_risk_data: Churn risk data for customer
        revenue_baseline: Revenue baseline for customer
        all_customers_baselines: All customers' baselines
        scoring_weights: Weight configuration
        max_gap_amount: Maximum gap amount for normalization

    Returns:
        Gap dictionary with scores added
    """
    customer_id = gap.get("customer_id", "")

    # Calculate individual scores
    revenue_impact = calculate_revenue_impact_score(gap, max_gap_amount)
    churn_risk = calculate_churn_risk_score(gap, churn_risk_data)
    customer_value = calculate_customer_value_score(
        customer_id,
        revenue_baseline,
        all_customers_baselines
    )
    recovery_probability = calculate_recovery_probability_score(
        gap,
        revenue_baseline
    )

    # Calculate weighted final score
    final_score = (
        revenue_impact * scoring_weights.get("revenue_impact", 0.35) +
        churn_risk * scoring_weights.get("churn_risk", 0.30) +
        customer_value * scoring_weights.get("customer_value", 0.20) +
        recovery_probability * scoring_weights.get("recovery_probability", 0.15)
    )

    # Add scores to gap
    scored_gap = gap.copy()
    scored_gap.update({
        "revenue_impact_score": revenue_impact,
        "churn_risk_score": churn_risk,
        "customer_value_score": customer_value,
        "recovery_probability_score": recovery_probability,
        "final_score": round(final_score, 2)
    })

    return scored_gap


def score_all_gaps(
    revenue_gaps: List[Dict[str, Any]],
    churn_risk_customers: List[Dict[str, Any]],
    customer_revenue_baseline: Dict[str, Dict[str, Any]],
    scoring_weights: Dict[str, float]
) -> List[Dict[str, Any]]:
    """
    Score all gaps.

    Args:
        revenue_gaps: List of all detected gaps
        churn_risk_customers: List of churn risk customers
        customer_revenue_baseline: Revenue baselines for all customers
        scoring_weights: Weight configuration

    Returns:
        List of scored gaps
    """
    # Build churn risk lookup
    churn_risk_lookup = {
        risk["customer_id"]: risk
        for risk in churn_risk_customers
    }

    # Find max gap amount for normalization
    if revenue_gaps:
        max_gap_amount = max(abs(gap.get("gap_amount", 0.0)) for gap in revenue_gaps)
        if max_gap_amount == 0:
            max_gap_amount = 1000.0  # Default
    else:
        max_gap_amount = 1000.0

    # Score each gap
    scored_gaps = []
    for gap in revenue_gaps:
        customer_id = gap.get("customer_id", "")
        revenue_baseline = customer_revenue_baseline.get(customer_id, {})
        churn_risk_data = churn_risk_lookup.get(customer_id)

        scored_gap = score_gap(
            gap,
            churn_risk_data,
            revenue_baseline,
            customer_revenue_baseline,
            scoring_weights,
            max_gap_amount
        )

        scored_gaps.append(scored_gap)

    return scored_gaps


def rank_gaps(
    scored_gaps: List[Dict[str, Any]],
    top_n: int = 20
) -> Dict[str, Any]:
    """
    Rank gaps by final score and select top N.

    Args:
        scored_gaps: List of scored gaps
        top_n: Number of top gaps to select

    Returns:
        Dictionary with ranked_gaps, top_priority_gaps, and summary
    """
    # Sort by final_score (descending)
    ranked = sorted(
        scored_gaps,
        key=lambda x: x.get("final_score", 0.0),
        reverse=True
    )

    # Select top N
    top_priority = ranked[:top_n]

    # Calculate summary metrics
    total_customers_analyzed = len(set(gap.get("customer_id") for gap in scored_gaps))
    customers_with_gaps = total_customers_analyzed
    total_revenue_gap = sum(abs(gap.get("gap_amount", 0.0)) for gap in scored_gaps)

    # Count by severity
    high_priority = sum(1 for gap in top_priority if gap.get("severity") == "high")
    medium_priority = sum(1 for gap in top_priority if gap.get("severity") == "medium")
    low_priority = sum(1 for gap in top_priority if gap.get("severity") == "low")

    # Estimate potential recovery revenue (sum of gaps for top N)
    potential_recovery_revenue = sum(abs(gap.get("gap_amount", 0.0)) for gap in top_priority)

    summary = {
        "total_customers_analyzed": total_customers_analyzed,
        "customers_with_gaps": customers_with_gaps,
        "total_revenue_gap": round(total_revenue_gap, 2),
        "high_priority_gaps": high_priority,
        "medium_priority_gaps": medium_priority,
        "low_priority_gaps": low_priority,
        "potential_recovery_revenue": round(potential_recovery_revenue, 2)
    }

    return {
        "ranked_gaps": ranked,
        "top_priority_gaps": top_priority,
        "gap_summary": summary
    }



# Scoring Node

In [ ]:
def scoring_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Scoring Node: Score all gaps by priority.

    Scores gaps using:
    - Revenue impact (35%)
    - Churn risk (30%)
    - Customer value (20%)
    - Recovery probability (15%)
    """
    errors = state.get("errors", [])
    revenue_gaps = state.get("revenue_gaps", [])
    churn_risk_customers = state.get("churn_risk_customers", [])
    customer_revenue_baseline = state.get("customer_revenue_baseline")

    if not revenue_gaps:
        return {
            "scored_gaps": [],
            "errors": errors
        }

    if not customer_revenue_baseline:
        return {
            "errors": errors + ["scoring_node: customer_revenue_baseline is required"]
        }

    # Get scoring weights from state or config
    from config import PredictiveRevenueGapConfig
    config = PredictiveRevenueGapConfig()

    scoring_weights = state.get("scoring_weights") or config.scoring_weights

    try:
        # Score all gaps
        scored_gaps = score_all_gaps(
            revenue_gaps,
            churn_risk_customers,
            customer_revenue_baseline,
            scoring_weights
        )

        return {
            "scored_gaps": scored_gaps,
            "errors": errors
        }

    except Exception as e:
        return {
            "errors": errors + [f"scoring_node: Unexpected error - {str(e)}"]
        }


def ranking_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Ranking Node: Rank gaps and select top N.

    Ranks gaps by final score and selects top priority gaps.
    """
    errors = state.get("errors", [])
    scored_gaps = state.get("scored_gaps", [])

    if not scored_gaps:
        return {
            "ranked_gaps": [],
            "top_priority_gaps": [],
            "gap_summary": {
                "total_customers_analyzed": 0,
                "customers_with_gaps": 0,
                "total_revenue_gap": 0.0,
                "high_priority_gaps": 0,
                "medium_priority_gaps": 0,
                "low_priority_gaps": 0,
                "potential_recovery_revenue": 0.0
            },
            "errors": errors
        }

    # Get top_n from state or config
    from config import PredictiveRevenueGapConfig
    config = PredictiveRevenueGapConfig()

    top_n = state.get("top_n_gaps") or config.top_n_gaps

    try:
        # Rank gaps
        ranking_result = rank_gaps(scored_gaps, top_n=top_n)

        return {
            "ranked_gaps": ranking_result["ranked_gaps"],
            "top_priority_gaps": ranking_result["top_priority_gaps"],
            "gap_summary": ranking_result["gap_summary"],
            "errors": errors
        }

    except Exception as e:
        return {
            "errors": errors + [f"ranking_node: Unexpected error - {str(e)}"]
        }


# Tests for Scoring Utilities

In [ ]:
"""
Tests for Scoring Utilities

Testing Phase 5: Scoring utilities before building the nodes
"""

import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path(__file__).parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

from agents.revenue_gap_orchestrator.utilities.scoring import (
    calculate_revenue_impact_score,
    calculate_churn_risk_score,
    calculate_customer_value_score,
    calculate_recovery_probability_score,
    score_gap,
    score_all_gaps,
    rank_gaps
)


def test_calculate_revenue_impact_score():
    """Test revenue impact score calculation"""
    gap = {"gap_amount": -500.0}

    score = calculate_revenue_impact_score(gap, max_gap_amount=1000.0)

    assert 0.0 <= score <= 10.0
    assert score == 5.0  # 500/1000 * 10 = 5.0
    print("✅ Calculate revenue impact score test passed")


def test_calculate_churn_risk_score_zero_spend():
    """Test churn risk score for zero spend gap"""
    gap = {
        "gap_type": "zero_spend",
        "weeks_at_risk": 2
    }

    score = calculate_churn_risk_score(gap)

    assert 0.0 <= score <= 10.0
    assert score >= 7.0  # Zero spend should be high risk
    print("✅ Calculate churn risk score (zero spend) test passed")


def test_calculate_churn_risk_score_with_data():
    """Test churn risk score with churn risk data"""
    gap = {"gap_type": "declining_revenue"}
    churn_risk_data = {"churn_risk_score": 0.8}

    score = calculate_churn_risk_score(gap, churn_risk_data)

    assert score == 8.0  # 0.8 * 10 = 8.0
    print("✅ Calculate churn risk score (with data) test passed")


def test_calculate_customer_value_score():
    """Test customer value score calculation"""
    customer_id = "1"
    revenue_baseline = {"total_revenue": 500.0}
    all_customers_baselines = {
        "1": {"total_revenue": 500.0},
        "2": {"total_revenue": 300.0},
        "3": {"total_revenue": 200.0},
        "4": {"total_revenue": 100.0},
    }

    score = calculate_customer_value_score(
        customer_id,
        revenue_baseline,
        all_customers_baselines
    )

    assert 0.0 <= score <= 10.0
    assert score >= 8.0  # Top customer should be high value
    print("✅ Calculate customer value score test passed")


def test_calculate_recovery_probability_score():
    """Test recovery probability score"""
    gap = {
        "gap_type": "below_baseline",
        "weeks_at_risk": 1
    }
    revenue_baseline = {"revenue_trend": "stable"}

    score = calculate_recovery_probability_score(gap, revenue_baseline)

    assert 0.0 <= score <= 10.0
    assert score >= 5.0  # Below baseline should have moderate recovery probability
    print("✅ Calculate recovery probability score test passed")


def test_score_gap():
    """Test scoring a complete gap"""
    gap = {
        "customer_id": "1",
        "gap_type": "declining_revenue",
        "gap_amount": -200.0,
        "gap_percentage": -25.0,
        "severity": "medium",
        "weeks_at_risk": 2
    }

    churn_risk_data = {"churn_risk_score": 0.6}
    revenue_baseline = {
        "total_revenue": 400.0,
        "revenue_trend": "declining"
    }
    all_customers_baselines = {
        "1": revenue_baseline,
        "2": {"total_revenue": 200.0}
    }
    scoring_weights = {
        "revenue_impact": 0.35,
        "churn_risk": 0.30,
        "customer_value": 0.20,
        "recovery_probability": 0.15
    }

    scored = score_gap(
        gap,
        churn_risk_data,
        revenue_baseline,
        all_customers_baselines,
        scoring_weights,
        max_gap_amount=1000.0
    )

    assert "revenue_impact_score" in scored
    assert "churn_risk_score" in scored
    assert "customer_value_score" in scored
    assert "recovery_probability_score" in scored
    assert "final_score" in scored
    assert 0.0 <= scored["final_score"] <= 10.0
    print("✅ Score gap test passed")


def test_score_all_gaps():
    """Test scoring all gaps"""
    revenue_gaps = [
        {
            "customer_id": "1",
            "gap_type": "declining_revenue",
            "gap_amount": -200.0,
            "weeks_at_risk": 2
        },
        {
            "customer_id": "2",
            "gap_type": "zero_spend",
            "gap_amount": -100.0,
            "weeks_at_risk": 2
        }
    ]

    churn_risk_customers = [
        {"customer_id": "1", "churn_risk_score": 0.6},
        {"customer_id": "2", "churn_risk_score": 0.8}
    ]

    customer_revenue_baseline = {
        "1": {"total_revenue": 400.0, "revenue_trend": "declining"},
        "2": {"total_revenue": 200.0, "revenue_trend": "stable"}
    }

    scoring_weights = {
        "revenue_impact": 0.35,
        "churn_risk": 0.30,
        "customer_value": 0.20,
        "recovery_probability": 0.15
    }

    scored = score_all_gaps(
        revenue_gaps,
        churn_risk_customers,
        customer_revenue_baseline,
        scoring_weights
    )

    assert len(scored) == 2
    assert all("final_score" in gap for gap in scored)
    print("✅ Score all gaps test passed")


def test_rank_gaps():
    """Test ranking gaps"""
    scored_gaps = [
        {"customer_id": "1", "final_score": 8.5, "gap_amount": -200.0, "severity": "high"},
        {"customer_id": "2", "final_score": 7.0, "gap_amount": -150.0, "severity": "medium"},
        {"customer_id": "3", "final_score": 5.5, "gap_amount": -100.0, "severity": "low"},
        {"customer_id": "4", "final_score": 6.0, "gap_amount": -120.0, "severity": "medium"},
    ]

    result = rank_gaps(scored_gaps, top_n=2)

    assert "ranked_gaps" in result
    assert "top_priority_gaps" in result
    assert "gap_summary" in result

    assert len(result["ranked_gaps"]) == 4
    assert len(result["top_priority_gaps"]) == 2
    assert result["top_priority_gaps"][0]["final_score"] == 8.5  # Highest first
    assert result["top_priority_gaps"][1]["final_score"] == 7.0

    # Check summary
    summary = result["gap_summary"]
    assert "total_customers_analyzed" in summary
    assert "total_revenue_gap" in summary
    assert "high_priority_gaps" in summary
    print("✅ Rank gaps test passed")


if __name__ == "__main__":
    print("Testing Scoring Utilities...\n")

    test_calculate_revenue_impact_score()
    test_calculate_churn_risk_score_zero_spend()
    test_calculate_churn_risk_score_with_data()
    test_calculate_customer_value_score()
    test_calculate_recovery_probability_score()
    test_score_gap()
    test_score_all_gaps()
    test_rank_gaps()

    print("\n✅ All scoring utility tests passed!")



In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_034_Predictive_Revenue_Gap_Orchestrator % python3 tests/test_scoring_utilities.py
Testing Scoring Utilities...

✅ Calculate revenue impact score test passed
✅ Calculate churn risk score (zero spend) test passed
✅ Calculate churn risk score (with data) test passed
✅ Calculate customer value score test passed
✅ Calculate recovery probability score test passed
✅ Score gap test passed
✅ Score all gaps test passed
✅ Rank gaps test passed

✅ All scoring utility tests passed!